In [ ]:
sentences = [
            'i love my dog',
            'I, love my cat',
            'You love my dog'
]

tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'love': 1, 'my': 2, 'i': 3, 'dog': 4, 'cat': 5, 'you': 6}


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
Tokenizer?

keras_preprocessing.text.Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
            'i love my dog',
            'I love my cat',
            'You love my dog',
             'Do you think my dog is amazing?'
]

tokenizer = Tokenizer(num_words=100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}


In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences

[[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]]

In [ ]:
padded = pad_sequences(sequences, maxlen=5)
print(padded)

[[ 0  5  3  2  4]
 [ 0  5  3  2  7]
 [ 0  6  3  2  4]
 [ 9  2  4 10 11]]


In [ ]:
padded = pad_sequences(sequences, maxlen=5, padding='post')
padded

array([[ 5,  3,  2,  4,  0],
       [ 5,  3,  2,  7,  0],
       [ 6,  3,  2,  4,  0],
       [ 9,  2,  4, 10, 11]], dtype=int32)

In [ ]:
padded = pad_sequences(sequences, maxlen=5, padding='post', truncating='post')
padded

array([[5, 3, 2, 4, 0],
       [5, 3, 2, 7, 0],
       [6, 3, 2, 4, 0],
       [8, 6, 9, 2, 4]], dtype=int32)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json
import json
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

--2021-07-23 05:24:12--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 173.194.210.128, 173.194.211.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2021-07-23 05:24:12 (168 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [ ]:
len(sentences)

26709

In [ ]:
vocab_size = 20000
tokenizer = Tokenizer(num_words = vocab_size , oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
len(word_index)

29657

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post', truncating='post')
padded.shape

(26709, 40)

In [ ]:
padded[0]

array([  308, 15115,   679,  3337,  2298,    48,   382,  2576, 15116,
           6,  2577,  8434,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0], dtype=int32)

In [35]:
import numpy as np
from sklearn.model_selection import train_test_split

In [33]:
type(labels[0])
labels = np.array(labels)

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential


In [ ]:
embedding_dim = 100
vocab_size = vocab_size
max_length = 40

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 100)           2000000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 606       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 2,000,613
Trainable params: 2,000,613
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded, labels, epochs=30)

Epoch 1/30
835/835 [==============================] - 18s 20ms/step - loss: 0.4985 - accuracy: 0.7484
Epoch 2/30
835/835 [==============================] - 17s 20ms/step - loss: 0.2670 - accuracy: 0.8918
Epoch 3/30
835/835 [==============================] - 16s 20ms/step - loss: 0.1914 - accuracy: 0.9278
Epoch 4/30
835/835 [==============================] - 17s 20ms/step - loss: 0.1446 - accuracy: 0.9479
Epoch 5/30
835/835 [==============================] - 17s 20ms/step - loss: 0.1134 - accuracy: 0.9616
Epoch 6/30
835/835 [==============================] - 17s 20ms/step - loss: 0.0878 - accuracy: 0.9716
Epoch 7/30
835/835 [==============================] - 17s 20ms/step - loss: 0.0698 - accuracy: 0.9775
Epoch 8/30
835/835 [==============================] - 17s 20ms/step - loss: 0.0561 - accuracy: 0.9821
Epoch 9/30
835/835 [==============================] - 17s 20ms/step - loss: 0.0454 - accuracy: 0.9867
Epoch 10/30
835/835 [==============================] - 17s 20ms/step - loss: 0.035